<a href="https://colab.research.google.com/github/JonNData/naive_bayes/blob/master/notebooks/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive Bayes
Multinomial 
Need a class.  
Fit and predict methods.  


*   Fit will get the histogram frequency, and train the model, update attributes.
-- Need a feature vector, row is obs, columns are features, numerical
*   Predict will classify the observation  

  

What's the input? Needs to be categorical but that can be easily arranged.


## Quick Example with Balance Scale data

In [2]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
headers = ['class_name', 'left-weight', 'left-distance', 'right-weight', 'right-distance']

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/balance-scale/balance-scale.data', names=headers)
df

In [ ]:
train, test = train_test_split(df, test_size = 0.2)

In [ ]:
X_train = train.drop(columns='class_name')
X_test = test.drop(columns='class_name')

y_train = train['class_name'].apply(lambda x: 0 if x=="L" else 1) 
y_test = test['class_name'].apply(lambda x: 0 if x=="L" else 1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

In [ ]:
y_pred = mnb.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)

## Now to make the class


In [ ]:
import numpy as np

In [ ]:
class GaussBayes:
  def __init__(self):
    pass
  def fit(self, X_train, y_train):
    # Get information about the array
    num_obs, num_features = X_train.shape
    
    # Get array of unique targets
    self.u_classes = np.unique(y_train)
    num_classes = len(self.u_classes)

    # intialize mean, variance, priors with correct size
    self.X_train_mean = np.zeros((num_classes, num_features), dtype=np.float64)
    self.var = np.zeros((num_classes, num_features), dtype=np.float64)
    self.priors = np.zeros(num_classes, dtype=np.float64)

    # Calculate
    for i,cls in enumerate(self.u_classes):
      # specify only values where the class is the same as the target
      X_train_cls = X_train[cls==y_train]
      # the mean from this class and all columns
      self.X_train_mean[cls,:] = X_train_cls.mean(axis=0)
      self.var[cls,:] = X_train_cls.var(axis=0)
      self.priors[cls] = X_train_cls.shape[0] / float(num_obs) # how often this class is occuring

  def predict(self, X_test):
    y_pred = [self._predict(x) for x in X_test]
    return y_pred

  def _predict(self, X_test):
    posteriors =  []

    for i, cls in enumerate(self.u_classes):
      prior = np.log(self.priors[i])
      class_conditional = np.sum(np.log(self._prob_density(i, X_test)))
      posterior =  prior + class_conditional
      posteriors.append(posterior)
    
    return self.u_classes[np.argmax(posteriors)] # argmax gives index of max value

  def _prob_density(self, class_idx, x):
      mean = self.X_train_mean[class_idx]
      var = self.var[class_idx]
      numerator = np.exp(- (x-mean)**2 / (2 * var))
      denominator = np.sqrt(2 * np.pi * var)
      return numerator / denominator

In [ ]:
gb = GaussBayes()
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)

In [ ]:
class NaiveBayes:
    def __init__(self, X, y):
        self.num_examples, self.num_features = X.shape
        self.num_classes = len(np.unique(y))
        self.eps = 1e-6

    def fit(self, X, y):
        self.classes_mean = {}
        self.classes_variance = {}
        self.classes_prior = {}

        for c in range(self.num_classes):
            X_c = X[y == c]

            self.classes_mean[str(c)] = np.mean(X_c, axis=0)
            self.classes_variance[str(c)] = np.var(X_c, axis=0)
            self.classes_prior[str(c)] = X_c.shape[0] / X.shape[0]

    def predict(self, X):
        probs = np.zeros((self.num_examples, self.num_classes))

        for c in range(self.num_classes):
            prior = self.classes_prior[str(c)]
            probs_c = self.density_function(
                X, self.classes_mean[str(c)], self.classes_variance[str(c)]
            )
            probs[:, c] = probs_c + np.log(prior)

        return np.argmax(probs, 1)

    def density_function(self, x, mean, sigma):
        # Calculate probability from Gaussian density function
        const = -self.num_features / 2 * np.log(2 * np.pi) - 0.5 * np.sum(
            np.log(sigma + self.eps)
        )
        probs = 0.5 * np.sum(np.power(x - mean, 2) / (sigma + self.eps), 1)
        return const - probs


In [ ]:
X_test.drop(X_test.tail(-1).index, inplace=True)
y_test.drop(X_test.tail(-1).index, inplace=True)
nb = NaiveBayes(X_train, y_train)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)
# Problem here is that test must be the same shape....

In [ ]:
class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors =  np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)
            
        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]
            

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

In [ ]:
nb = NaiveBayes()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)

In [ ]:
class MultiNayes:
    """
    Multinomial Naive Bayes algorithm.
    Paramaters
    ----------
    alpha : float, default=1.0
        Smoothing paramater, can be set to smaller values
        (0 for no smoothing)
    """

    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.fitted = False

    def label_binarizer(self, y, classes=None, bin_labels=None):
        """convert labels into an array of shape
           (length of y, number of classes). This
           will assist in getting the log priors and probabilities"""
        if classes is None:
            classes = np.unique(y)
            bin_labels = np.zeros((y.shape[0], classes.shape[0]))
            self.classes = classes
            self.bin_labels = bin_labels

        if bin_labels.shape[0] < 1:
            return None

        x = np.where(classes == y[0])
        bin_labels[0][x] = 1

        return self.label_binarizer(y[1:], classes, bin_labels[1:])

    def fit(self, X, y):
        # if X is not np.ndarray, convert from csr with `toarray()`
        if type(X) is not np.ndarray:
            X = X.to_numpy()

        self.label_binarizer(y)

        n_classes = self.classes.shape[0]
        n_features = X.shape[1]

        # initialize counter arrays
        self.class_count = np.zeros(n_classes)
        self.feature_count = np.zeros((n_classes, n_features))

        # count classes and features by getting
        # dot product of transposed binary labels
        # they are automatically separated into their
        # appropriate arrays
        self.feature_count += np.dot(self.bin_labels.T, X)
        self.class_count += self.bin_labels.sum(axis=0)

        # add smoothing
        if self.alpha > 0.0:
            self.feature_count += self.alpha
            smoothed_class_count = self.feature_count.sum(axis=1)

            # get conditional log probabilities
            self.feat_log_probs = (np.log(self.feature_count) -
                                   np.log(smoothed_class_count.reshape(-1, 1)))
        else:
            print(
                f"Alpha is {self.alpha}. A value this small will cause "
                "result in errors when feature count is 0"
            )
            self.feat_log_probs = np.log(
                                    self.feature_count /
                                    self.feature_count
                                    .sum(axis=1)
                                    .reshape(-1, 1)
                                  )

        # get log priors
        self.class_log_priors = (np.log(self.class_count) -
                                 np.log(self.class_count
                                 .sum(axis=0)
                                 .reshape(-1, 1)))

        self.fitted = True

    def predict(self, X):
        """Predict target from features of X"""

        # check if model has fit data
        if not self.fitted:
            print("The classifier has not yet "
                  "been fit. Not executing predict")

        if type(X) is not np.ndarray:
            X = X.toarray()

        scores = np.dot(X, self.feat_log_probs.T) + self.class_log_priors

        predictions = self.classes[np.argmax(scores, axis=1)]

        return predictions

    def accuracy(self, y_pred, y):
        points = (y_pred == y).astype(int)
        score = points.sum() / points.shape[0]

        return score

In [ ]:
clf = MultiNayes()


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)

## Bernoulli

In [ ]:
def tokenize(text):
  text = text.lower() # convert to lowercase
  stop_chars = '''0123456789!()-[]{};:'"\,<>./?@#$%^&*_~'''


  # To take input from the user
  # my_str = input("Enter a string: ")

  # remove punctuation from the string
  tokens = ""
  for char in text:
    if char not in stop_chars:
        tokens += char

  return tokens.split()

def count_words(training_set):
  """ 
  input is (message, category) where the category is 0 or 1
  output is counts = {word1: [num_cat1, num_cat2], word2: [num_cat1, num_cat2]...}
  """
  counts = {}
  for message, category in training_set:
    for word in tokenize(message):
      if word not in counts:
        counts[word] = [0,0]
      counts[word][category] += 1
  return counts

def word_probabilities(counts, total_cat_1, total_cat_2, k=0.5):
  """turn the word_counts into a list of triplets
  w, p(w | cat1) and p(w | cat2)"""
  word, 
  return [(w,
  (spam + k) / (total_spams + 2 * k),
  (non_spam + k) / (total_non_spams + 2 * k))
  for w, (spam, non_spam) in counts.items()]

def spam_probability(word_probs, message):
  message_words = tokenize(message)
  log_prob_if_spam = log_prob_if_not_spam = 0.0
  # iterate through each word in our vocabulary
  for word, prob_if_spam, prob_if_not_spam in word_probs:
    # if *word* appears in the message,
    # add the log probability of seeing it
    if word in message_words:
      log_prob_if_spam += math.log(prob_if_spam)
      log_prob_if_not_spam += math.log(prob_if_not_spam)
    # if *word* doesn't appear in the message
  # add the log probability of _not_ seeing it
  # which is log(1 - probability of seeing it)
    else:
      log_prob_if_spam += math.log(1.0 - prob_if_spam)
      log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

  prob_if_spam = math.exp(log_prob_if_spam)
  prob_if_not_spam = math.exp(log_prob_if_not_spam)
  return prob_if_spam / (prob_if_spam + prob_if_not_spam)




In [ ]:
class NaiveBayesClassifier:
  def __init__(self, k=0.5):
    self.k = k
    self.word_probs = []
  def train(self, training_set):
    # count spam and non-spam messages
    num_spams = len([is_spam
    for message, is_spam in training_set
    if is_spam])
    num_non_spams = len(training_set) - num_spams
    # run training data through our "pipeline"
    word_counts = count_words(training_set)
    self.word_probs = word_probabilities(word_counts,
    num_spams,
    num_non_spams,
    self.k)
  def classify(self, message):
    return spam_probability(self.word_probs, message)

In [ ]:
nbc = NaiveBayesClassifier()

In [ ]:
"""
Let's say input is [("sample text1", label1), ("sample text2", label2)]
"""

In [ ]:
text = "will this work? The rights among men can be strenuous..."
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''


# To take input from the user
# my_str = input("Enter a string: ")

# remove punctuation from the string
no_punct = ""
for char in text:
   if char not in punctuations:
       no_punct = no_punct + char
no_punct.split()

['will',
 'this',
 'work',
 'The',
 'rights',
 'among',
 'men',
 'can',
 'be',
 'strenuous']

In [ ]:
def tokenize1(text):
  text = text.lower() # convert to lowercase
  stop_chars = '''0123456789!()-[]{};:'"\,<>./?@#$%^&*_~'''


  # To take input from the user
  # my_str = input("Enter a string: ")

  # remove punctuation from the string
  tokens = ""
  for char in text:
    if char not in stop_chars:
        tokens += char

  return tokens.split()

In [ ]:
def count_words(training_set):
  """ 
  input is (message, category) where the category is 0 or 1
  output is counts = {word1: [num_cat1, num_cat2], word2: [num_cat1, num_cat2]...}
  """
  counts = {}
  for message, category in training_set:
    for word in tokenize1(message):
      if word not in counts:
        counts[word] = [0,0]
      counts[word][category] += 1
  return counts

def word_probabilities(counts, total_cat_1, total_cat_2, k=0.5):
  """turn the word_counts into a list of triplets
  w, p(w | cat1) and p(w | cat2)"""

  return [(w,
  (cat1 + k) / (total_cat_1 + 2 * k),
  (total_cat_2 + k) / (total_cat_2 + 2 * k))
  for w, (cat1, cat2) in counts.items()]

def cat_probability(word_probs, message):
  message_words = tokenize1(message)
  log_prob_cat_1 = log_prob_cat_2 = 0.0
  # iterate through each word in our vocabulary
  for word, prob_cat_1, prob_cat_2 in word_probs:
    # if *word* appears in the message,
    # add the log probability of seeing it
    if word in message_words:
      log_prob_cat_1 += np.log(prob_cat_1)
      log_prob_cat_2 += np.log(prob_cat_2)
    # if *word* doesn't appear in the message
  # add the log probability of _not_ seeing it
  # which is log(1 - probability of seeing it)
    else:
      log_prob_cat_1 += np.log(1.0 - prob_cat_1)
      log_prob_cat_2 += np.log(1.0 - prob_cat_2)

  prob_cat_1 = np.exp(log_prob_cat_1)
  prob_cat_2 = np.exp(log_prob_cat_2)
  return prob_cat_1 / (prob_cat_1 + prob_cat_2)

In [ ]:
trainin = [
           ("""Our second function will count the words in a labeled training set of messages. We’ll have
it return a dictionary whose keys are words, and whose values are two-element lists
[spam_count, non_spam_count] corresponding to how many times we saw that word in
both spam and nonspam messages:
""", 0),
("""Our next step is to turn these counts into estimated probabilities using the smoothing we
described before. Our function will return a list of triplets containing each word, the
probability of seeing that word in a spam message, and the probability of seeing that word
in a nonspam message:""", 1),
("""The last piece is to use these word probabilities (and our Naive Bayes assumptions) to
assign probabilities to messages""", 0)
]
# count spam and non-spam messages
num_cat_1 = len([cat
for message, cat in trainin
if cat==0])
num_cat_2 = len(trainin) - num_cat_1
word_counts = count_words(trainin)

word_probs = word_probabilities(word_counts, num_cat_1, num_cat_2)
message = "The last piece is to use these word probabilities"
cat_probability(word_probs, message)

1.0

In [ ]:
count_words(trainin)
word_probs
# here we have the word and the probability. 
# remember we have to switch to counts of words instead of binary
# TODO multiply a prior probability of each category by the probability of each 
# word in the message given it was in each category. p(cat1) * p(w|cat1) * p(w1|cat1)
# etc. same for cat2
# then compare and predict the higher value.

[('twoelement', 0.5, 0.75),
 ('count', 0.5, 0.75),
 ('that', 0.5, 0.75),
 ('corresponding', 0.5, 0.75),
 ('many', 0.5, 0.75),
 ('we', 0.5, 0.75),
 ('values', 0.5, 0.75),
 ('saw', 0.5, 0.75),
 ('it', 0.5, 0.75),
 ('of', 0.5, 0.75),
 ('function', 0.5, 0.75),
 ('our', 0.8333333333333334, 0.75),
 ('messages', 0.8333333333333334, 0.75),
 ('keys', 0.5, 0.75),
 ('a', 0.5, 0.75),
 ('times', 0.5, 0.75),
 ('and', 0.8333333333333334, 0.75),
 ('have', 0.5, 0.75),
 ('in', 0.5, 0.75),
 ('training', 0.5, 0.75),
 ('set', 0.5, 0.75),
 ('spam', 0.5, 0.75),
 ('second', 0.5, 0.75),
 ('dictionary', 0.5, 0.75),
 ('word', 0.8333333333333334, 0.75),
 ('whose', 0.5, 0.75),
 ('labeled', 0.5, 0.75),
 ('the', 0.8333333333333334, 0.75),
 ('return', 0.5, 0.75),
 ('are', 0.5, 0.75),
 ('both', 0.5, 0.75),
 ('lists', 0.5, 0.75),
 ('will', 0.5, 0.75),
 ('words', 0.5, 0.75),
 ('how', 0.5, 0.75),
 ('nonspam', 0.5, 0.75),
 ('nonspamcount', 0.5, 0.75),
 ('to', 0.8333333333333334, 0.75),
 ('spamcount', 0.5, 0.75),
 ('we’ll'